## Instructions + Grading

+ To get full marks for each participation worksheet, you must successfully answer at least 40% of all autograded questions. In this worksheet, 40% is 11 questions.

+ Autograded questions are easily identifiable through their labelling as **QUESTION**. Any other instructions that prompt the student to write code are activities, which are not graded and thus do not contribute to marks - but do contribute to the workflow of the worksheet!

## Attributions

The following resources were used as inspiration in the creation of this worksheet:

+ [Swirl R Programming Tutorial](https://swirlstats.com/scn/rprog.html)
+ [Palmer Penguins R Package](https://github.com/hadley/palmerpenguins)
+ [RD4S Data Transformation](https://r4ds.had.co.nz/transform.html)

# Part A: Getting started with your data

**Part A of this worksheet reviews the concepts learned in Class 4: Data Wrangling I.**

What's the first thing that you should do when you're starting a project with a new dataset? Having a coffee is a reasonable answer, but before that, you should **look at the data**. This may sound obvious, but a common mistake is to dive into the analysis too early before being familiar with the data - only to have to go back to the start when something goes wrong and you can't quite figure out why. Some of the questions you may want to ask are:

+ What is the format of the data?
+ What are the dimensions?
+ Are there missing data?

In part A of this worksheet, you will learn how to answer these questions and more using dplyr.

Load the `palmerpenguins`, `tidyverse`, `testthat`, and `digest` packages. Hint: `suppressPackageStartupMessages()`!
    - This loads `dplyr`, too.

In [ ]:
# load your packages here:
library(FILL_THIS_IN)
library(FILL_THIS_IN)
library(FILL_THIS_IN)
library(FILL_THIS_in)

[Palmer penguins](https://github.com/hadley/palmerpenguins) is an R data package created by Allison Horst. Data were collected and made available by Dr. Kristen Gorman and the Palmer Station, Antarctica LTER, a member of the Long Term Ecological Research Network. The dataset that we will be using is stored in a variable called "penguins". It is a subset of the "penguins_raw" dataset, also included in this R package. Let's have a look at it.

In [ ]:
head(penguins)

head() returns the first 6 rows of a dataframe, instead of printing all the data to screen.

## What is the format of the data?

Let's begin by checking the class of the **penguins** variable. This will give us a clue about the overall structure of the data.

In [ ]:
class(FILL_THIS_IN)

As you can see, the function returns 3 classes: "tbl_df", "tbl", and "data.frame". A dataframe is the default class for data read into R. Tibbles ("tbl" and "tbl_df") are a modern take on data frames, but slightly tweaked to work better in the tidyverse. For now, you don’t need to worry about the differences; we’ll come back to tibbles later. The dataset that we are working with was originally a data.frame that has been coerced into a tibble, which is why multiple class names are returned by the class() function.

## What are the dimensions?

There are two functions that we can use to see exactly how many rows (observations) and columns (variables) we're dealing with. dim() is the base R option, and glimpse() is the dplyr flavour, which gives us some more information besides the row and column number. Give both a try!

In [ ]:
dim(FILL_THIS_IN)
glimpse(FILL_THIS_IN)

There are more functions that you can use to further explore the dimensions, such as nrow(), ncol(), colnames() or rownames(), but we won't be looking into those.

**QUESTION 1.0**

In the dim() function, what is the first number that you see?

1: rows
2: columns

In [ ]:
# answer1.0 <- youranswer

### BEGIN SOLUTION
answer1.0 <- 1
### END SOLUTION

In [ ]:
test_that("Question 1.0", {
    expect_equal(digest(as.numeric(answer1.0)), "6717f2823d3202449301145073ab8719")
})

## Exploring further: Five cornerstones of dplyr

So far, we've **looked** at our dataset. It's time to **work with** it! Prior to creating any models, or using visualization to gain more insights about our data, it is common to tweak the data in some ways to make it a little easier to work with. For example, you may need to rename some variables, reorder observations, or even create some new variables from your existing ones!

As explained in depth in the [R4DS Data Transformation chapter](https://r4ds.had.co.nz/transform.html), there are five key dplyr functions that allow you to solve the vast majority of data manipulation tasks:

+ Pick variables by their names (select())
+ Pick observations by their values (filter())
+ Reorder the rows (arrange())
+ Create new variables with functions of existing variables (mutate())
+ Collapse many values down to a single summary (summarise())

### `select()` 

*A brief interlude on naming things:* Names are important. Jenny Bryan has some excellent [slides](https://speakerdeck.com/jennybc/how-to-name-files) for naming things in a way that is human readable *and* machine readable. Don't worry too much about it for this worksheet, but do keep it in mind as it helps with *reproducibility*. 

A quick tip that you can put into practice: use *Pascal case* - creating names by concatenating capitalized words, such as PenguinsSubset, or PenguinsTidy. If names get too long, remove vowels! For example, PngnSubset, or PngnTidy instead.

**QUESTION 1.1**

In the next few questions, you will practice using the dplyr verb select() to pick and modify variables by their names. Make a data frame containing the columns `species`, `island`, `sex` from the penguins data, in that order.

In [ ]:
# answer1.1 <- select(penguins, FILL_THIS_IN)

### BEGIN SOLUTION
answer1.1 <- select(penguins, c(species, island, sex))
### END SOLUTION

In [ ]:
test_that("Question 1.1", {
    expect_equal(digest(as_tibble(answer1.1)), "0df5cac5070ec518519a6f2781f4e01f")
})

# not sure if this is correct, a bit confused as to how to write tests for tibbles... i did digest::digest(answer1.1) manually and copied the code

Out of the following options, what would be the best name for the object that you just created above (currently stored in "answer1.1")?

1: _penguin_subset
2: penguins
3: 2penguin
4: PngnSub

In [ ]:
# answer1.1a <- youranswer

### BEGIN SOLUTION
answer1.1a <- 4
### END SOLUTION

In [ ]:
test_that("Question 1.1a", {
    expect_equal(digest(as.numeric(answer1.1a)), "dbc09cba9fe2583fb01d63c70e1555a8")
})

**QUESTION 1.2**

Select all variables, from `bill_length_mm` to `body_mass_g`. Of course, you could do it this way...

In [ ]:
# This will work:
select(penguins, bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g)

But there is a better way to do it! Which do you think would work?

1: select(penguins, body_mass_g:bill_length_mm)
2: select(penguins, c(body_mass_g::bill_length_mm))
3: select(penguins, bill_length_mm:body_mass_g)
4: select(penguins, bill_length_mm::body_mass_g)

In [ ]:
# answer1.2 <- youranswer

### BEGIN SOLUTION
answer1.2 <- 3
### END SOLUTION

In [ ]:
test_that("Question 1.2", {
    expect_equal(digest(as.numeric(answer1.2)), "e5b57f323c7b3719bbaaf9f96b260d39")
})

Now run the code yourself to see the output!

In [ ]:
# your code here

**QUESTION 1.3**

You're doing a great job. Keep it up! Now, select all variables, except `island`. How would you write this code?

1: select(penguins, -c("island"))
2: select(penguins, -island)
3: select(penguins, -("island"))

In [ ]:
# answer1.3 <- youranswer

### BEGIN SOLUTION
answer1.3 <- 2
### END SOLUTION

In [ ]:
test_that("Question 1.3", {
    expect_equal(digest(as.numeric(answer1.3)), "db8e490a925a60e62212cefc7674ca02")
})

Your turn to write the code and see the output.

In [ ]:
# your code here

**QUESTION 1.4**

Put `year` first. Hint: use the `everything()` function.

In [ ]:
# answer1.4 <- select(penguins, FILL_THIS_IN, FILL_THIS_IN)

### BEGIN SOLUTION
answer1.4 <- select(penguins, year, everything())
### END SOLUTION

In [ ]:
# test that there are 344 rows and 8 columns, which are in the following order: year, species, island, bill_length_mm, bill_depth_mm, bill_depth_mm, flipper_length_mm, body_mass_g, sex
test_that("Question 1.4", {
    expect_equal(digest(answer1.4), "a07a1cdcb64726866df3d525811a9bf6")
})

**QUESTION 1.5**

Rename `flipper_length_mm` to `length_flipper_mm`.

In [ ]:
# answer1.5 <- rename(FILL_THIS_IN, FILL_THIS_IN)

### BEGIN SOLUTION
answer1.5 <- rename(penguins, c(length_flipper_mm = "flipper_length_mm"))
### END SOLUTION

In [ ]:
# check that there are 344 rows and 8 columns: year, species, island, bill_length_mm, bill_depth_mm, bill_depth_mm, length_flipper_mm, body_mass_g, sex
test_that("Question 1.5", {
  expect_equal(digest(dim(answer1.5)), 'd095e682a86f7f16404b7f8dd5f3d676')
  expect_equal(digest(colnames(answer1.5)), 'ef6a2aaa40de41c0b11ad2f6888d5ce6')
})

### `filter()` 

So far, we've practiced picking variables by their name with select(). But how about picking observations? This is where filter() comes in.

**QUESTION 1.6**

Pick penguins with body mass greater than 3600 g.

In [ ]:
# answer1.6 <- filter(FILL_THIS_IN, FILL_THIS_IN)

### BEGIN SOLUTION
answer1.6 <- filter(penguins, body_mass_g > 3600)
### END SOLUTION

In [ ]:
# test for an object that is 245 rows by 8 columns and that there are no entries in the body_mass_g column that is <3600
test_that("Question 1.6", {
  expect_equal(digest(dim(answer1.6)), '0f80c9cad929bf5de5ae34e0d50cb60d')
  
  # Is this ok?
  expect_equal(sum(pull(answer1.6, body_mass_g) < 3600), 0)
})

In question 1.6 above, you've created a subset of the `penguins` dataset by filtering for those penguins that have a body mass greater than 3600 g. Let's do a quick check to see how many penguins meet that threshold by comparing the dimensions of the `penguins` dataset and your subset, `answer1.6`. Do you remember how to check the dimensions of a dataframe? Hint: there are two different ways to do this. 

In [ ]:
# compare
## check dimensions of `penguins`
## check dimensions of `answer1.6`

As you can see, in filtering down to penguins with a body mass greater than 3600g, we have lost about 100 rows (observations). However, `answer1.6` doesn't seem like an informative name for this new dataset that you've created from `penguins`. Let's rename it to something else.

In [ ]:
penguins3600 <- answer1.6

**QUESTION 1.7**

From your "new" dataset, take only data from penguins located in the Biscoe island.

In [ ]:
# answer1.7 <- filter(FILL_THIS_IN, FILL_THIS_IN)

### BEGIN SOLUTION 
answer1.7 <- filter(penguins3600, island == "Biscoe")
### END SOLUTION

In [ ]:
# test for an object that is 148 rows by 8 columns and that all entries in the island column are "Biscoe"
test_that("Question 1.7", {
  expect_equal(digest(dim(answer1.7)), "92ac01cd2e8809faceb1f7a283cd935f")
  
  a <- unique(pull(answer1.7, island))
  expect_length(a, 1)
  expect_equal(digest(a), "2522027d230e3dfe02d8b6eba1fd73e1")
})

**QUESTION 1.8**

Repeat the task from question 1.7, but take data from islands Torgersen and Dream. Now that you've practiced with dplyr verbs quite a bit, you don't need as many prompts to answer! Hint: When you want to select more than one island, you use %in% instead of =.

In [ ]:
# answer1.8 <- youranswer

### BEGIN SOLUTION
answer1.8 <- filter(penguins3600, island %in% c("Torgersen", "Dream"))
### END SOLUTION

In [ ]:
# test for an object that is 97 rows by 8 columns and that 28 entries in the island column are "Torgersen", and 69 of them are "Dream"
test_that("Question 1.8", {
  expect_equal(digest(dim(answer1.8)), "b207bbce54bb47be51e7ba7b56d24bc2")
  expect_equal(sum(pull(answer1.8, island) == "Torgersen"), 28)
  expect_equal(sum(pull(answer1.8, island) == "Dream"), 69)
})

### `arrange()` 

Arrange() allows you to rearrange rows. Let's give it a try!

**QUESTION 1.9**

Order `penguins` by year, in ascending order.

In [ ]:
# answer1.9 <- arrange(FILL_THIS_IN, FILL_THIS_IN)

### BEGIN SOLUTION
answer1.9 <- arrange(penguins, year)
### END SOLUTION

In [ ]:
# test that year column is in ascending order, not sure if this is possible?

# vy: I think this will work, but check please
expect_sorted <- function(object) {
  act <- quasi_label(rlang::enquo(object), arg = "object")
  expect(
    !is.unsorted(act$val),
    sprintf("%s not sorted", act$lab)
  )
  invisible(act$val)
}

test_that("Question 1.9", {
    expect_sorted(pull(answer1.9, year))
})

**QUESTION 1.10**

Great work! Order `penguins` by year, in descending order. Hint: there is a function that allows you to order a variable in descending order...

In [ ]:
# answer1.10 <- arrange(FILL_THIS_IN, FILL_THIS_IN)

### BEGIN SOLUTION
answer1.10 <- arrange(penguins, desc(year))
### END SOLUTION

In [ ]:
# test that year column is in descending order, not sure if this is possible?
test_that("Question 1.10", {
    expect_sorted(pull(answer1.10, year) %>% 
                    rev())
})

**QUESTION 1.11**

Order `penguins` by year, then by body_mass_g. Use ascending order in both cases.

In [ ]:
# answer1.11 <- arrange(FILL_THIS_IN, FILL_THIS_IN, FILL_THIS_IN)

### BEGIN SOLUTION
answer1.11 <- arrange(penguins, year, body_mass_g)
### END SOLUTION

In [ ]:
# again, not sure if it's possible to test the order of the columns... what else could we test for?
test_that("Question 1.11", {
  
  # works for year
  expect_sorted(pull(answer1.11, year))
  
  # problem is now for body_mass_g for each year...
  # this is my attempt
  answer1.11_list <- answer1.11 %>% 
    group_by(year) %>% 
    group_split()
  
  expect_length(answer1.11_list, 3)
  expect_sorted(answer1.11_list[[1]] %>% pull(body_mass_g) %>% na.omit())
  expect_sorted(answer1.11_list[[2]] %>% pull(body_mass_g) %>% na.omit())
  expect_sorted(answer1.11_list[[3]] %>% pull(body_mass_g) %>% na.omit())
})

### Piping, `%>%` 

So far, we've been using dplyr verbs by inputting the dataset that we want to work on as the first argument of the function (e.g. select(**penguins**, year)). This is fine when you're using a single verb, i.e. you only want to filter observations, or select variables. However, more often than not you will want to do several tasks at once; such as filtering penguins with a certain body mass, and simultaneously ordering those penguins by year. Here is where piping (%>%) comes in.

Think of `%>%` as the word "then"!

Let's see an example. Here I want to combine `select()` with `arrange()`.

This is how I could do it by *nesting* the two function calls. I am selecting variables year, species, island, and body_mass_g, while simultaneously arranging by year.

In [ ]:
arrange(select(penguins, year, species, island, body_mass_g), year)

However, that seems a little hard to read. Now using pipes:

In [ ]:
penguins %>%
  select(year, species, island, body_mass_g) %>%
  arrange(year)

### `mutate()` 

The `mutate()` function _creates_ new columns in the tibble by transforming other variables. Like `select()`, `filter()`, and `arrange()`, the `mutate()` function also takes a tibble as its first argument, and returns a tibble. 

The general syntax is: mutate(tibble, NEW_COLUMN_NAME = CALCULATION).

**QUESTION 1.12**

Make a new column with body mass in kg, named `body_mass_kg`, *and* rearrange the tibble so that `body_mass_kg` goes after `body_mass_g` and before `sex`. Hint: within select(), use R's `:` operator to select all variables from `species` to `body_mass_g`.

In [ ]:
# answer1.12 <- 
# penguins %>%
# mutate(FILL_THIS_IN = FILL_THIS_IN) %>%
# select(FILL_THIS_IN, FILL_THIS_IN, FILL_THIS_IN, FILL_THIS_IN)

### BEGIN SOLUTION
answer1.12 <- 
penguins %>%
  mutate(body_mass_kg = body_mass_g / 1000) %>%
  select(species:body_mass_g, body_mass_kg, sex, year)
### END SOLUTION

In [ ]:
# test for an object that is 344 by 9, and where the column order is:
# [1] "species"           "island"            "bill_length_mm"   
# [4] "bill_depth_mm"     "flipper_length_mm" "body_mass_g"      
# [7] "body_mass_kg"      "sex"               "year" 

test_that("Question 1.12", {
  expect_equal(digest(dim(answer1.12)), "9e9457527d068c2333ea8fd598e07f13")
  expect_equal(digest(colnames(answer1.12)), "d7121e41fe934232c1c45dc425365040")
  
  # tests that body_mass_kg was correctly made
  expect_equal(na.omit(answer1.12$body_mass_kg / answer1.12$body_mass_g) %>% digest,
               "cdfbfd4da65e3575a474558218939055")
})

Have a look at `answer1.12` to see what the changes you made look like.

In [ ]:
# your code here

Notice the backwards compatibility! No need for loops! Now, try the same thing, but with `transmute` (drops all other variables). 

In [ ]:
# penguins %>%
#  transmute(FILL_THIS_IN = FILL_THIS_IN)

### `summarise()`

The last key verb is summarise(). It collapses a data frame to a single row:

In [ ]:
summarise(penguins, body_mass_mean = mean(body_mass_g, na.rm = TRUE))

*From R4DS Data Transformation:* 

> "summarise() is not terribly useful unless we pair it with group_by(). This changes the unit of analysis from the complete dataset to individual groups. Then, when you use the dplyr verbs on a grouped data frame they’ll be automatically applied "by group"." 

For example, if we applied exactly the same code to a data frame grouped by island, we get the average body mass per island:

In [ ]:
penguins %>%
  group_by(island) %>%
  summarise(body_mass_mean = mean(body_mass_g, na.rm = TRUE))

**QUESTION 1.13**

Try the same thing as in the example above, but this time grouping the penguins by year, and then island.

In [ ]:
# answer1.13 <- youranswer

### BEGIN SOLUTION
answer1.13 <- 
  penguins %>%
  group_by(year, island) %>%
  summarise(body_mass_mean = mean(body_mass_g, na.rm = TRUE))
### END SOLUTION

In [ ]:
# test that output is 9 by 3, and columns are: "year"           "island"         "body_mass_mean"
test_that("Question 1.13", {
  expect_equal(digest(dim(answer1.13)), "f4885de1726d18557bd43d769cc0ae26")
  expect_equal(digest(colnames(answer1.13)), "ba0c85220a5fa5222cac937acb2f94c2")
})

## Creating tibbles

Throughout Part A, we have been working with a tibble, `penguins`. Remember that when we ran 'class()' on `penguins`, we could see that it was a dataframe that had been coerced to a tibble, which is a unifying feature of the tidyverse.

Suppose that you have a dataframe that you want to coerce to a tibble. To do this, you can use "as_tibble()". R comes with a few built-in datasets, one of which is `mtcars`. Check the class of `mtcars` below.

In [ ]:
# your code here

As you can see, mtcars is a dataframe. Now, coerce it to a tibble with as_tibble().

In [ ]:
as_tibble(mtcars)

You can read more about tibbles in the [R4DS Tibble Chapter](https://r4ds.had.co.nz/tibbles.html#creating-tibbles).

## Time to review what we've learned!

**QUESTION 1.14**

Load the package `gapminder` with library(). Check the class of the `gapminder` dataset.

In [ ]:
# library(FILL_THIS_IN)
# class(FILL_THIS_IN)

As you can see, it has already been coerced to a tibble.

Take all countries in Europe that have a GDP per capita greater than 10000, and select all variables except `gdpPercap`, using pipes. (Hint: use `-`).

In [ ]:
# answer1.14 <- FILL_THIS_IN %>%
#   filter(FILL_THIS_IN > 10000) %>%
#   FILL_THIS_IN(-FILL_THIS_IN)

### BEGIN SOLUTION
answer1.14 <-
  gapminder %>%
  filter(gdpPercap > 10000) %>%
  select(-gdpPercap)
### END SOLUTION

In [ ]:
# test that 392 by 5 and that all gdpPercap > 10000
test_that("Question 1.14", {
  expect_equal(digest(dim(answer1.14)), "10e7af552b77b2008cc3a6465100596d")
  
  # it's hard to verify that all gdpPercap > 10000, since we asked to drop that column
  expect_equal(digest(answer1.14), "3218056845cc9759df4f0098613e4428")
})

**QUESTION 1.15**

Coerce the `mtcars` data frame to a tibble, and take all columns that start with the letter "d". 
    - Hint: take a look at the "Select helpers" documentation by running the following code: `?tidyselect::select_helpers`.

In [ ]:
# answer1.15 <- #youranswer

### BEGIN SOLUTION
answer1.15 <- as_tibble(mtcars) %>% select(starts_with("d"))
### END SOLUTION

In [ ]:
# test for a 32 by 2 tibble where columns are "disp", "drat" in that order
test_that("Question 1.15", {
  expect_equal(digest(dim(answer1.15)), "ea1df69d6a59227894d1d4330f9bfab8")
  expect_equal(digest(colnames(answer1.15)), "0956954d01fe74c59c1f16850b7e874f")
})

This exercise is from [r-exercises](https://www.r-exercises.com/2017/10/19/dplyr-basic-functions-exercises/).

## Part B: Visualize your data

**Part B of this worksheet reviews the concepts learned in Class 5: ggplot2 I.**

## Preliminary

Run this code chunk to load the required packages:

In [ ]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(palmerpenguins))
suppressPackageStartupMessages(library(gapminder))
suppressPackageStartupMessages(library(scales))
suppressPackageStartupMessages(library(tsibble))

## Grammar of Graphics

Before we start, here is the [ggplot2 cheatsheet](https://github.com/rstudio/cheatsheets/blob/master/data-visualization-2.1.pdf). When you're coding on your own (i.e. working on your own project outside of class), cheatsheets are one of the most common resources that you'll continuously revisit.

Consider the following plot. Don't concern yourself with the code at this point.

In [ ]:
gapminder %>% 
  filter(year == 2007) %>% 
  mutate(continent = fct_infreq(continent)) %>% 
  ggplot(aes(continent)) +
  geom_bar() +
  theme_bw()

**QUESTION 2.0**

Fill in the seven grammar components for this plot. Choose the best option:

1:

| Grammar Component     | Specification |
|-----------------------|---------------|
| __data__              | `gapminder` |
| __aesthetic mapping__ | `continent` |
| __geometric object__  | `bars` |
| scale                 | `logarithmic` |
| statistical transform | `none` |
| coordinate system     | `rectangular` |
| facetting             | `none` |

2:

| Grammar Component     | Specification |
|-----------------------|---------------|
| __data__              | `gapminder` |
| __aesthetic mapping__ | `continent` |
| __geometric object__  | `bars` |
| scale                 | `linear` |
| statistical transform | `none` |
| coordinate system     | `rectangular` |
| facetting             | `none` |

3:

| Grammar Component     | Specification |
|-----------------------|---------------|
| __data__              | `gapminder` |
| __aesthetic mapping__ | `year` |
| __geometric object__  | `bars` |
| scale                 | `linear` |
| statistical transform | `none` |
| coordinate system     | `rectangular` |
| facetting             | `none` |


In [ ]:
# answer2.0 <- youranswerhere

### BEGIN SOLUTION
answer2.0 <- 2
### END SOLUTION

In [ ]:
test_that("Question 2.0", {
    expect_equal(digest(as.numeric(answer2.0)), "db8e490a925a60e62212cefc7674ca02")
})

Look at this plot instead.

In [ ]:
ggplot(gapminder, aes(gdpPercap, lifeExp)) +
  geom_point(alpha = 0.1) +
  scale_x_log10("GDP per capita", labels = scales::dollar_format()) +
  theme_bw() +
  ylab("Life Expectancy")

**QUESTION 2.1**

What is the aesthetic mapping in this plot?

1: alpha = 0.1
2: x: gdpPercap, y: lifeExp
3: none
4: "Life Expectancy"

In [ ]:
# answer2.1 <- youranswer

### BEGIN SOLUTION
answer2.1 <- 2
### END SOLUTION

In [ ]:
test_that("Question 2.1", {
    expect_equal(digest(as.numeric(answer2.1)), "db8e490a925a60e62212cefc7674ca02")
})

## `ggplot2` Syntax 

The following is a tsibble (a special type of tibble containing time series data, which we'll see more of later), stored in the variable `mauna`, of CO$_2$ concentrations collected monthly at the Mauna Loa station.

Execute this code to store the data in `mauna`:

In [ ]:
(mauna <- tsibble::as_tsibble(co2) %>% 
   rename(month = index, conc = value))

**QUESTION 2.2**

Produce a line chart showing the concentration over time. Specifically, the plot should have the following grammar components:

| Grammar Component     | Specification |
|-----------------------|---------------|
| __data__              | `mauna` |
| __aesthetic mapping__ | x: month, y: conc |
| __geometric object__  | lines |
| scale                 | linear |
| statistical transform | none |
| coordinate system     | rectangular |
| facetting             | none |

Fill in the blanks to obtain the plot:

In [ ]:
# answer2.2 <- ggplot(FILL_THIS_IN, aes(FILL_THIS_IN, FILL_THIS_IN)) +
# FILL_THIS_IN()

### BEGIN SOLUTION
answer2.2 <- ggplot(mauna, aes(x = month, y = conc)) + geom_line()
### END SOLUTION

In [ ]:
# test that the aesthetics of the plot are aes(x = month, y = conc) and geom_line()
test_that("Question 2.2", {
  
  expect_equal(digest(answer2.2$mapping), "58410547f02ab682a5231e8b7898486d")
  
  # tests that the first geom layer is geom_line
    expect_equal(digest(answer2.2$layers[[1]]$geom),
                 "e6be55794c0cd28ca01a12ac74dd0f5f")
})

In [ ]:
# print plot to screen here

**QUESTION 2.3**

It turns out that you're allowed to specify the aesthetic mappings in a `geom` layer instead of, or in addition to, in the `ggplot()` function, with the following rules:

- Aesthetics appearing in a `geom` layer apply only to that layer.
- If there are conflicting aesthetics in both the `ggplot()` function and the `geom` layer, the `geom` layer takes precedence.

The following code mistakenly puts the month variable on the y-axis. Fill in the `FILL_THIS_IN` so that you still obtain the same result as above.

In [ ]:
# answer2.3 <-
# ggplot(mauna, aes(y = month)) +
# geom_point(aes(FILL_THIS_IN))

### BEGIN SOLUTION 
answer2.3 <-
ggplot(mauna, aes(y = month)) +
  geom_point(aes(x = month))
### END SOLUTION

In [ ]:
# test for the plot aesthetics
test_that("Question 2.3", {
  expect_equal(digest(answer2.3$mapping), "")
})

**QUESTION 2.4**

You can store the output of the plot in a variable, too. The plot that you created in 2.3 is stored in the variable `answer2.2`. Add a layer to `answer2.2` that adds green points to the plot. 

In [ ]:
#answer2.4 <-
#answer2.3 +
#  FILL_THIS_IN(colour = FILL_THIS_IN)

### BEGIN SOLUTION
answer2.4 <-
answer2.3 +
  geom_point(colour = "green")
### END SOLUTION

In [ ]:
# test for the plot aesthetics
test_that("Question 2.4", {
  expect_equal(digest(answer2.4$mapping), "")
})

Have a look at the plot!

In [ ]:
# print the plot to screen

**QUESTION 2.5**

Can you fix what is wrong with the following code?

In [ ]:
# answer2.5 <-
# ggplot(gapminder) +
#  geom_point(x = gdpPercap, y = lifeExp, alpha = 0.1)

### BEGIN SOLUTION
answer2.5 <-
ggplot(gapminder, aes(x = gdpPercap, y = lifeExp)) +
  geom_point(alpha = 0.1)
### END SOLUTION

In [ ]:
# test for the plot aesthetics
test_that("Question 2.5", {
  expect_equal(digest(answer2.5$mapping), "6b0fe1934ff2907a629cc8b6d8581367")
})

In [ ]:
# print to screen

**QUESTION 2.6**

So you're a ggplot2 pro? Then, let's see this plot adapted to polar coordinates. Specifically:

- angle is month (January through December)
- radius is CO$_2$ concentration

The plot should look like a spiral, or concentric circles. 

In [ ]:
#answer2.6 <-
#ggplot(mauna, aes(x = FILL_THIS_IN, y = FILL_THIS_IN)) +
#  geom_point() +
#  coord_polar()

### BEGIN SOLUTION
answer2.6 <-
ggplot(mauna, aes(x = month, y = conc)) +
  geom_point() +
  coord_polar()
### END SOLUTION

In [ ]:
test_that("Question 2.6", {
  
  # test for the plot aesthetics
  expect_equal(digest(answer2.6$mapping), "58410547f02ab682a5231e8b7898486d")
  
  # test for polar coordinates
  expect_equal(digest(answer2.6$coordinates), "e75662f6b6f38ad49266f787531f1ec5")
})

Remember the `penguins` dataset that we worked with in Part A of the worksheet? Let's try visualizing some of that data.

**QUESTION 2.7**

There are many questions that you may want to ask about the `penguins` dataset. Often, exploring relationships between variables is one of the first steps when exploring your data. Scatterplots are a great way to visualize this. 

Let's explore the relationship between flipper length and body mass (if any!). I have provided the "scaffold" of the ggplot2 recipe, now it's your turn to fill it in. Use the following parameters:

- Flipper length should be on the x axis, and body mass should be on the y axis.
- Color by species.
- Set the size of the points to 3, and the alpha (transparency) to 0.8.
- 

In [ ]:
# answer2.7 <- 
#  ggplot(data = FILL_THIS_IN, aes(x = FILL_THIS_IN, y = FILL_THIS_IN)) +
#  geom_point(aes(color = FILL_THIS_IN),
#             size = FILL_THIS_IN,
#             alpha = FILL_THIS_IN) 

### BEGIN SOLUTION
answer2.7 <- 
  ggplot(data = penguins,
                    aes(x = flipper_length_mm, y = body_mass_g)) +
  geom_point(aes(color = species),
             size = 3,
             alpha = 0.8) 
### END SOLUTION

In [ ]:
# test for the plot aesthetics
test_that("Question 2.7", {
  
  # test for the plot aesthetics
  expect_equal(digest(answer2.7$mapping), "687de3963fbc964bd4dcbcc28484a1cd")
  
  # test for geom_point parameters
  expect_equal(digest(answer2.7$layers[[1]]), "6760c6d3a827d2c6797258061927e443")
})

Now, print `answer2.7` to screen to see your plot!

In [ ]:
# print to screen

**QUESTION 2.8**

Try the same as in 2.7 (flipper length vs. body mass), but this time coloring by island! See if you can write the code without looking at the last question.

In [ ]:
# answer2.8 <- youranswer

### BEGIN SOLUTION
answer2.8 <- 
  ggplot(data = penguins,
                    aes(x = flipper_length_mm, y = body_mass_g)) +
  geom_point(aes(color = island),
             size = 3,
             alpha = 0.8) 
### END SOLUTION

In [ ]:
# test for the plot aesthetics 
test_that("Question 2.8", {
  
  # test for the plot aesthetics
  expect_equal(digest(answer2.8$mapping), "687de3963fbc964bd4dcbcc28484a1cd")
  
  # test for geom_point parameters
  expect_equal(digest(answer2.8$layers[[1]]), "7e13e5b0683e061bcce7601d3a512a68")
})

Print `answer2.7` to screen to see the plot!

In [ ]:
# your code here

**QUESTION 2.9**

Suppose you want to visualize the number of penguins per island. It seems that a bar chart could be useful for this purpose. Give it a try! Hint: bar charts only need specifying one axis (use the x axis).

In [ ]:
# answer2.9 <- ggplot(penguins, aes(FILL_THIS_IN)) + FILL_THIS_IN()

### BEGIN SOLUTION
answer2.9 <- ggplot(penguins, aes(x = island)) + geom_bar()
### END SOLUTION

In [ ]:
# test for the plot aesthetics 
test_that("Question 2.9", {
  
  # test for the plot aesthetics
  expect_equal(digest(answer2.9$mapping), "f85cbd0c4331d25180383917c3112e53")
  
  # test for geom_bar 
  expect_equal(digest(answer2.9$layers[[1]]), "682b1d788de6421850a6663bf8c6e1f4")
})

Print the plot to screen to see the penguin distribution across different islands.

In [ ]:
# your code here

Try plotting the same graph, but specifying the y axis coordinate instead. What does it look like?

In [ ]:
# your code here

Great job! Bar plots can be plotted in either way. Here is another handy way to flip the coordinates, instead of plotting the graph again.

In [ ]:
# take answer2.9, which stores the vertical graph, and add coord_flip()
# answer2.9 + coord_flip()

You get the same result!

**QUESTION 2.10**

Maybe you want to explore the distribution of body mass. Exploring the distribution is one of the most important things when looking at data; different models have different assumptions, and the distribution of data is usually one of those assumptions. No need to worry about models for now, but it's interesting to know ways in which you can explore distribution visually! Histograms are useful for this purpose.

In [ ]:
# answer2.10 <-
# ggplot(FILL_THIS_IN, aes(x = FILL_THIS_IN)) +
#  geom_histogram()

### BEGIN SOLUTION
answer2.10 <-
ggplot(penguins, aes(x = body_mass_g)) +
  geom_histogram()
### END SOLUTION

In [ ]:
# test for plot aesthetics
test_that("Question 2.10", {
  
  # test for the plot aesthetics
  expect_equal(digest(answer2.10$mapping), "9278ccffa7b7fee0acb5e87bd5b004a0")
  
  # test for geom_histogram
  expect_equal(digest(answer2.10$layers[[1]]), "c0d17114c83620da7aff8273841e5113")
})

In [ ]:
# your code here

Visualizing the distribution of body mass across all data is great, but a more interesting question may be what the body mass distribution is by species. We can achieve this easily by adding to the code:

In [ ]:
ggplot(penguins, aes(x = body_mass_g, fill = species)) +
  geom_histogram()

Great! We've added some more information, but is there something we could do to make the graph more readable? Let's start by changing the binwidth:

In [ ]:
ggplot(penguins, aes(x = body_mass_g, fill = species)) +
  geom_histogram(binwidth = 50)

**QUESTION 2.11**

Stacked histograms still look a little awkward. When we want to visualize distributions of the data by variable in a single graph, density plots look better in terms of variables not obscuring one another. Try it yourself! Hint: Use `color` or `colour` instead of `fill` for aesthetic mapping. 

In [ ]:
# answer2.11 <- ggplot(COMPLETE_THE_CODE) + geom_density()

### BEGIN SOLUTION
answer2.11 <- 
ggplot(penguins, aes(x = body_mass_g, color = species)) +
  geom_density()
### END SOLUTION

In [ ]:
# test for plot aesthetics
test_that("Question 2.11", {
  
  # test for the plot aesthetics
  expect_equal(digest(answer2.11$mapping), "e56a9c19c487767537579dc2458e308e")
  
  # test for geom_histogram
  expect_equal(digest(answer2.11$layers[[1]]), "5cb8099528fc10ef3d8778b001005d52")
})

See what the plot looks like now.

In [ ]:
# print to screen

**QUESTION 2.12**

Let's look at two more types of plot to once again explore the relationship between body mass and penguin species. Stripplots are univariate scatterplots, and they are a useful way of visualizing how, in this case, body mass is spread in each species. Specify "species" as x, and "body_mass_g" as y.

In [ ]:
# answer2.12 <-
#  ggplot(FILL_THIS_IN, aes(COMPLETE_THE_CODE)) + geom_jitter()

### BEGIN SOLUTION
answer2.12 <- 
  ggplot(penguins, aes(x = species, y = body_mass_g)) + geom_jitter()
### END SOLUTION

In [ ]:
# test for plot aesthetics
test_that("Question 2.12", {
  
  # test for the plot aesthetics
  expect_equal(digest(answer2.12$mapping), "9ee4270e4d1c7cfcd518ca9416b8a152")
  
  # test for geom_histogram
  expect_equal(digest(answer2.12$layers[[1]]), "966b8b64d86f3d044eda56575dec2ddf")
})

**QUESTION 2.13**

A similar option is a boxplot. Hint: use geom_boxplot() instead.

In [ ]:
# answer2.13 <- youranswer

### BEGIN SOLUTION
answer2.13 <- 
  ggplot(penguins, aes(x = species, y = body_mass_g)) + geom_boxplot()
### END SOLUTION

In [ ]:
# test for plot aesthetics
test_that("Question 2.13", {
  
  # test for the plot aesthetics
  expect_equal(digest(answer2.13$mapping), "9ee4270e4d1c7cfcd518ca9416b8a152")
  
  # test for geom_histogram
  expect_equal(digest(answer2.13$layers[[1]]), "a4054159b57b244e746b594e0ef7e2f8")
})

Now, let's have a look at both plots.

In [ ]:
answer2.12
answer2.13

**Congratulations!** You've finished Part B of Worksheet 2, and with that, Worksheet 2 overall. Which also means that you've completed 3 weeks of STAT545A, and you're well on your way to being an R pro. Not quite done with practicing data viz with ggplot? We'll be diving deeper into plotting next week, but here are a few resources if you want to explore in the meantime:

+ [Jenny Bryan's ggplot2 tutorial](https://github.com/jennybc/ggplot2-tutorial). Has lots of examples, but less dialogue.
+ [R4DS Data Viz chapter](https://r4ds.had.co.nz/data-visualisation.html). With a comprehensive walk-through style.
+ [Chapter 2 of the ggplot book](http://webcat2.library.ubc.ca/vwebv/holdingsInfo?bibId=8489511). Even more detailed!

